# 2000 - 2016 Voting Data
## Imports

In [1]:
import pandas as pd

# Flipped Data for County

In [2]:
# read the dataframe
df = pd.read_csv('2000_2016_voting_data_import/countypres_2000-2016.csv')

In [3]:
# find the winners
dfw = df.groupby(['FIPS', 'year'])['candidatevotes'].max().to_frame()
df2 = dfw.merge(df, left_on=['FIPS', 'year'], right_on=['FIPS', 'year'])
df3 = df2[df2['candidatevotes_x'] <= df2['candidatevotes_y']]
df3 = df3.copy()
df3['winner_percentage'] = df3['candidatevotes_y']/df3['totalvotes']
winners = df3[['FIPS', 'year', 'candidate', 'winner_percentage']]


In [4]:
# get each candidate and their party
df_cand = df[['candidate', 'party']]
df_cand = df_cand.drop_duplicates()
df_cand

,candidate,party
0,Al Gore,democrat
1,George W. Bush,republican
2,Ralph Nader,green
3,Other,NaN
12464,John Kerry,democrat
21815,Barack Obama,democrat
21816,John McCain,republican
31167,Mitt Romney,republican
40517,Hillary Clinton,democrat
40518,Donald Trump,republican


In [5]:
# get each county and the FIPS and county
df_county_info = df[['FIPS', 'county', 'state']]
df_county_info = df_county_info .drop_duplicates()
df_county_info.head() 

,FIPS,county,state
0,1001.0,Autauga,Alabama
4,1003.0,Baldwin,Alabama
8,1005.0,Barbour,Alabama
12,1007.0,Bibb,Alabama
16,1009.0,Blount,Alabama


In [6]:
# Get the needed info and put into a weird form
# Unpack and put it into a weird form
winners2 = winners.pivot(index = 'FIPS', columns = 'year', values = 'candidate').reset_index()
winners3 =  winners.pivot(index = 'FIPS', columns = 'year', values = 'winner_percentage').reset_index()

winners4 = winners2.merge(winners3, left_on='FIPS', right_on='FIPS')

winners2 = winners4.rename(index=str, 
                   columns={
                       "2000_x": "2000_cand",
                       "2004_x": "2004_cand",
                       "2008_x": "2008_cand",
                       "2012_x": "2012_cand",
                       "2016_x": "2016_cand",
                       
                       "2000_y": "2000_percent",
                       "2004_y": "2004_percent",
                       "2008_y": "2008_percent",
                       "2012_y": "2012_percent",
                       "2016_y": "2016_percent"
                       
                   })
winners2.head()

year,FIPS,2000_cand,2004_cand,2008_cand,2012_cand,2016_cand,2000_percent,2004_percent,2008_percent,2012_percent,2016_percent
0,1001.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.696943,0.756735,0.736136,0.726183,0.727666
1,1003.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.723654,0.764152,0.752595,0.773583,0.765457
2,1005.0,Al Gore,George W. Bush,John McCain,Barack Obama,Donald Trump,0.499086,0.547369,0.504385,0.513685,0.520967
3,1007.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.601746,0.720000,0.724433,0.728266,0.764032
4,1009.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.704779,0.808501,0.840195,0.864659,0.893348


In [7]:
# rename and get the party for each year
for i in range(2000, 2020, 4):
    index = str(i) + "_cand"
    winners2 = winners2.merge(df_cand, left_on=index, right_on='candidate')
    winners2 = winners2.rename(index=str, 
                   columns={
                       "party": "party" + str(i),                 
                   })
    winners2 = winners2.drop(columns=['candidate'])

winners2.head()

,FIPS,2000_cand,2004_cand,2008_cand,2012_cand,2016_cand,2000_percent,2004_percent,2008_percent,2012_percent,2016_percent,party2000,party2004,party2008,party2012,party2016
0,1001.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.696943,0.756735,0.736136,0.726183,0.727666,republican,republican,republican,republican,republican
1,1003.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.723654,0.764152,0.752595,0.773583,0.765457,republican,republican,republican,republican,republican
2,1007.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.601746,0.720000,0.724433,0.728266,0.764032,republican,republican,republican,republican,republican
3,1009.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.704779,0.808501,0.840195,0.864659,0.893348,republican,republican,republican,republican,republican
4,1013.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.528899,0.591611,0.564940,0.535699,0.561269,republican,republican,republican,republican,republican


In [8]:
# store if there was a switch for this year
winners3 = winners2
for i in range(2000, 2016, 4):
    j = i + 4
    winners3['Switch_' + str(i) + "_" + str(j)] = winners3['party'+str(i)] !=  winners3['party'+str(j)]
winners3[winners3['Switch_2012_2016'] == True].head()
# len(winners3)

,FIPS,2000_cand,2004_cand,2008_cand,2012_cand,2016_cand,2000_percent,2004_percent,2008_percent,2012_percent,2016_percent,party2000,party2004,party2008,party2012,party2016,Switch_2000_2004,Switch_2004_2008,Switch_2008_2012,Switch_2012_2016
2423,2015.0,George W. Bush,George W. Bush,John McCain,Barack Obama,Donald Trump,0.417888,0.653117,0.742107,0.508913,0.506825,republican,republican,republican,democrat,republican,False,False,True,True
2424,2031.0,George W. Bush,George W. Bush,John McCain,Barack Obama,Donald Trump,0.557586,0.629011,0.628082,0.501232,0.551660,republican,republican,republican,democrat,republican,False,False,True,True
2425,2032.0,George W. Bush,George W. Bush,John McCain,Barack Obama,Donald Trump,0.692524,0.581975,0.557975,0.622376,0.503747,republican,republican,republican,democrat,republican,False,False,True,True
2426,2036.0,George W. Bush,George W. Bush,John McCain,Barack Obama,Donald Trump,0.539179,0.610865,0.629552,0.671963,0.539690,republican,republican,republican,democrat,republican,False,False,True,True
2427,8015.0,George W. Bush,George W. Bush,John McCain,Barack Obama,Donald Trump,0.565046,0.555936,0.491230,0.486140,0.479200,republican,republican,republican,democrat,republican,False,False,True,True


In [9]:
winners4 = winners3
winners4 = winners4.merge(df_county_info, left_on = 'FIPS', right_on = 'FIPS') 
winners4.head()

,FIPS,2000_cand,2004_cand,2008_cand,2012_cand,2016_cand,2000_percent,2004_percent,2008_percent,2012_percent,...,party2004,party2008,party2012,party2016,Switch_2000_2004,Switch_2004_2008,Switch_2008_2012,Switch_2012_2016,county,state
0,1001.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.696943,0.756735,0.736136,0.726183,...,republican,republican,republican,republican,False,False,False,False,Autauga,Alabama
1,1003.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.723654,0.764152,0.752595,0.773583,...,republican,republican,republican,republican,False,False,False,False,Baldwin,Alabama
2,1007.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.601746,0.720000,0.724433,0.728266,...,republican,republican,republican,republican,False,False,False,False,Bibb,Alabama
3,1009.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.704779,0.808501,0.840195,0.864659,...,republican,republican,republican,republican,False,False,False,False,Blount,Alabama
4,1013.0,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,0.528899,0.591611,0.564940,0.535699,...,republican,republican,republican,republican,False,False,False,False,Butler,Alabama


In [10]:
winners4.to_csv("2000_2016_voting_data_export/FP_county_winners.csv")

# Flipped Data for State

In [11]:
# read the state data
df = pd.read_csv('2000_2016_voting_data_import/1976-2016-president.csv')
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes
0,1976,Alabama,AL,1,63,41,US President,"Carter, Jimmy",democrat,False,659170,1182850,20171015,NaN
1,1976,Alabama,AL,1,63,41,US President,"Ford, Gerald",republican,False,504070,1182850,20171015,NaN
2,1976,Alabama,AL,1,63,41,US President,"Maddox, Lester",american independent party,False,9198,1182850,20171015,NaN
3,1976,Alabama,AL,1,63,41,US President,"Bubar, Benjamin """"Ben""""",prohibition,False,6669,1182850,20171015,NaN
4,1976,Alabama,AL,1,63,41,US President,"Hall, Gus",communist party use,False,1954,1182850,20171015,NaN


In [12]:
print("Number of total elections:", (1 + (2016-1976)//4) )
print("Number of states:", len(df['state'].unique()))



Number of total elections: 11
Number of states: 51


In [13]:
# find the winners
dfw = df.groupby(['state_po', 'year'])['candidatevotes'].max().to_frame()
df2 = dfw.merge(df, left_on=['state_po', 'year'], right_on=['state_po', 'year'])
df3 = df2[df2['candidatevotes_x'] <= df2['candidatevotes_y']]
# make sure that there's only one winner per year/state
assert(len(df3) == (11 * 51))
df3.head()
df3 = df3.copy()
df3['winner_percentage'] = df3['candidatevotes_y']/df3['totalvotes']
winners = df3[['state_po', 'year', 'candidate', 'winner_percentage']]
winners.head()

,state_po,year,candidate,winner_percentage
0,AK,1976,"Ford, Gerald",0.579046
4,AK,1980,"Reagan, Ronald",0.543482
9,AK,1984,"Reagan, Ronald",0.666540
13,AK,1988,"Bush, George H.W.",0.595909
19,AK,1992,"Bush, George H.W.",0.394575


In [14]:
# get each candidate and their party
df_cand = df[['candidate', 'party']]
df_cand = df_cand.drop_duplicates()

df_cand = df_cand[(df_cand['party'] == 'democrat') | (df_cand['party'] == 'republican')]
df_cand.head()

,candidate,party
0,"Carter, Jimmy",democrat
1,"Ford, Gerald",republican
332,"Reagan, Ronald",republican
697,"Mondale, Walter",democrat
1019,"Bush, George H.W.",republican


In [15]:
# Get the needed info and put into a weird form
winners = winners[winners['year'] >= 1996]
winners2 = winners.pivot(index = 'state_po', columns = 'year', values = 'candidate').reset_index()
winners3 =  winners.pivot(index = 'state_po', columns = 'year', values = 'winner_percentage').reset_index()

winners4 = winners2.merge(winners3, left_on='state_po', right_on='state_po')
winners2 = winners4.rename(index=str, 
                   columns={
                       "1996_x": "1996_cand",
                       "2000_x": "2000_cand",
                       "2004_x": "2004_cand",
                       "2008_x": "2008_cand",
                       "2012_x": "2012_cand",
                       "2016_x": "2016_cand",
                       
                       "1996_y": "1996_percent",
                       "2000_y": "2000_percent",
                       "2004_y": "2004_percent",
                       "2008_y": "2008_percent",
                       "2012_y": "2012_percent",
                       "2016_y": "2016_percent"
                       
                   })
winners2.head()

year,state_po,1996_cand,2000_cand,2004_cand,2008_cand,2012_cand,2016_cand,1996_percent,2000_percent,2004_percent,2008_percent,2012_percent,2016_percent
0,AK,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.508013,0.586210,0.610653,0.594245,0.548016,0.512815
1,AL,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.501218,0.564838,0.624607,0.603169,0.605458,0.620831
2,AR,"Clinton, Bill","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.537364,0.513072,0.543060,0.587159,0.605669,0.605741
3,AZ,"Clinton, Bill","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.465171,0.510211,0.548694,0.536352,0.536545,0.486716
4,CA,"Clinton, Bill","Gore, Al","Kerry, John","Obama, Barack H.","Obama, Barack H.","Clinton, Hillary",0.510989,0.534497,0.543056,0.610126,0.602390,0.617264


In [16]:
# rename and get the party for each year
for i in range(1996, 2020, 4):
    index = str(i) + "_cand"
    winners2 = winners2.merge(df_cand, left_on=index, right_on='candidate')
    winners2 = winners2.rename(index=str, 
                   columns={
                       "party": "party" + str(i),                 
                   })
    winners2 = winners2.drop(columns=['candidate'])

winners2.head()

,state_po,1996_cand,2000_cand,2004_cand,2008_cand,2012_cand,2016_cand,1996_percent,2000_percent,2004_percent,2008_percent,2012_percent,2016_percent,party1996,party2000,party2004,party2008,party2012,party2016
0,AK,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.508013,0.586210,0.610653,0.594245,0.548016,0.512815,republican,republican,republican,republican,republican,republican
1,AL,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.501218,0.564838,0.624607,0.603169,0.605458,0.620831,republican,republican,republican,republican,republican,republican
2,GA,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.470157,0.549596,0.579748,0.522045,0.533292,0.507713,republican,republican,republican,republican,republican,republican
3,ID,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.521841,0.671704,0.683909,0.615171,0.645298,0.592614,republican,republican,republican,republican,republican,republican
4,KS,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.542907,0.580417,0.620040,0.566123,0.597113,0.566546,republican,republican,republican,republican,republican,republican


In [17]:
# store if there was a switch for this year
winners3 = winners2
for i in range(1996, 2016, 4):
    j = i + 4
    winners3['Switch_' + str(i) + "_" + str(j)] = winners3['party'+str(i)] !=  winners3['party'+str(j)]
winners3[winners3['Switch_2012_2016'] == True].head()
winners3 = winners3.drop_duplicates()
winners3.head()

,state_po,1996_cand,2000_cand,2004_cand,2008_cand,2012_cand,2016_cand,1996_percent,2000_percent,2004_percent,...,party2000,party2004,party2008,party2012,party2016,Switch_1996_2000,Switch_2000_2004,Switch_2004_2008,Switch_2008_2012,Switch_2012_2016
0,AK,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.508013,0.586210,0.610653,...,republican,republican,republican,republican,republican,False,False,False,False,False
1,AL,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.501218,0.564838,0.624607,...,republican,republican,republican,republican,republican,False,False,False,False,False
2,GA,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.470157,0.549596,0.579748,...,republican,republican,republican,republican,republican,False,False,False,False,False
3,ID,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.521841,0.671704,0.683909,...,republican,republican,republican,republican,republican,False,False,False,False,False
4,KS,"Dole, Robert","Bush, George W.","Bush, George W.","McCain, John","Romney, Mitt","Trump, Donald J.",0.542907,0.580417,0.620040,...,republican,republican,republican,republican,republican,False,False,False,False,False


In [18]:
winners3[['state_po', 'party1996', 'party2000', 'party2004', 'party2008', 'party2012', 'party2016']]

,state_po,party1996,party2000,party2004,party2008,party2012,party2016
0,AK,republican,republican,republican,republican,republican,republican
1,AL,republican,republican,republican,republican,republican,republican
2,GA,republican,republican,republican,republican,republican,republican
3,ID,republican,republican,republican,republican,republican,republican
4,KS,republican,republican,republican,republican,republican,republican
5,MS,republican,republican,republican,republican,republican,republican
6,MT,republican,republican,republican,republican,republican,republican
7,ND,republican,republican,republican,republican,republican,republican
8,NE,republican,republican,republican,republican,republican,republican
9,OK,republican,republican,republican,republican,republican,republican


In [19]:
winners4 = winners3.drop(columns=['1996_cand', '1996_percent'])
winners4.to_csv("2000_2016_voting_data_export/FP_state_winners.csv")
for c in winners4.columns:
    print(c)


state_po
2000_cand
2004_cand
2008_cand
2012_cand
2016_cand
2000_percent
2004_percent
2008_percent
2012_percent
2016_percent
party1996
party2000
party2004
party2008
party2012
party2016
Switch_1996_2000
Switch_2000_2004
Switch_2004_2008
Switch_2008_2012
Switch_2012_2016


Here are the states that flopped in the 2016 election that we need to look for polling data on

In [20]:
winners4[winners4['Switch_2012_2016']][['state_po', 'party2016']]

,state_po,party2016
24,FL,republican
25,OH,republican
26,IA,republican
27,MI,republican
28,PA,republican
29,WI,republican
